# Instalação de pacotes

In [12]:
!powershell pip install \
            face_recognition \
            git+https://github.com/ageitgey/face_recognition_models \
            opencv-python \
            pillow \
            numpy \
            tqdm \
            qdrant-client \
            setuptools \
            ipywidgets \
            matplotlib \
            scikit-learn \
            scikit-image

  Cloning https://github.com/ageitgey/face_recognition_models to c:\users\despinxz\appdata\local\temp\pip-req-build-ht9vz2if
  Resolved https://github.com/ageitgey/face_recognition_models to commit e67de717267507d1e9246de95692eb8be736ab61
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   --------------------- ------------------ 7.1/12.9 MB 36.4 MB/s eta 0:00:01
   ---------------------------------------  12.8/12.9 MB 32.3 MB/s eta 0:00:01
   ---------------------------------------  12.8/12.9 MB 32.3 MB/s eta 0:00:01
   ---------------------------------------  12.8/12.9 MB 32.3 MB/s eta 0:00:01
   ---------------------------------------- 12

  Running command git clone --filter=blob:none --quiet https://github.com/ageitgey/face_recognition_models 'C:\Users\despinxz\AppData\Local\Temp\pip-req-build-ht9vz2if'


# Rodar QDrant

In [ ]:
!powershell docker run -p 6333:6333 -p 6334:6334 qdrant/qdrant

# Imports

In [13]:
import os
import uuid
import numpy as np
from tqdm import tqdm
import tkinter as tk
from tkinter import filedialog
import face_recognition
import face_recognition_models
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import Filter, SearchRequest
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn
import skimage

# Configurações

In [10]:
def cria_cliente():
    # Conectando ao Qdrant local
    client = QdrantClient("http://localhost:6333")
    return client

def cria_colecao(client, nome_colecao, tamanho_vetor):
    # Cria a coleção se não existir
    if nome_colecao not in [c.name for c in client.get_collections().collections]:
        client.recreate_collection(
            collection_name=nome_colecao,
            vectors_config=VectorParams(size=tamanho_vetor, distance=Distance.COSINE),
        )

# Códigos para processar imagens

### Gerar vetor de características por Local Binary Patterns

In [20]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

def extrai_caracteristicas_lbp(caminho_imagem):
    imagem = cv2.imread(caminho_imagem)
    # Converte imagem para tons de cinza
    imagem_cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

    metodo = 'uniform'
    vetor_final = []

    # Para radius 1, 2 e 3
    for raio in [1, 2, 3]:
        n_pontos = 8 * raio

        # Aplica o LBP
        lbp = local_binary_pattern(imagem_cinza, n_pontos, raio, metodo)

        # Número de bins: n_pontos + 2 para o método 'uniform'
        n_bins = n_pontos + 2
        hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)

        # Adiciona o histograma ao vetor final
        vetor_final.extend(hist)

    return np.array(vetor_final)


### Gerar vetor de características por descritores faciais

In [17]:
import face_recognition
import numpy as np
import cv2

def extrai_caracteristicas_landmarks(caminho_imagem):
    imagem = cv2.imread(caminho_imagem)
    # Converte para RGB (face_recognition usa RGB)
    imagem_rgb = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

    # Detecta landmarks faciais
    face_landmarks_list = face_recognition.face_landmarks(imagem_rgb)

    # Se não encontrou rosto, retorna vetor vazio
    if not face_landmarks_list:
        return np.zeros(136)  # 68 pontos * 2 coordenadas (x, y)

    # Pegamos os landmarks da primeira face detectada
    landmarks = face_landmarks_list[0]

    # Reorganiza os pontos na ordem correta (68 pontos padrão)
    pontos = []
    for key in ['chin', 'left_eyebrow', 'right_eyebrow', 'nose_bridge', 'nose_tip',
                'left_eye', 'right_eye', 'top_lip', 'bottom_lip']:
        for ponto in landmarks[key]:
            pontos.append(ponto)

    # Garante que temos sempre 68 pontos
    if len(pontos) < 68:
        pontos += [(0, 0)] * (68 - len(pontos))

    # Normaliza por largura e altura da imagem
    h, w = imagem.shape[:2]
    vetor = []
    for (x, y) in pontos[:68]:
        vetor.append(x / w)
        vetor.append(y / h)

    return np.array(vetor)


### Gerar vetor de características por CNN

In [18]:
def extrai_caracteristicas_cnn(caminho_imagem):
    imagem = face_recognition.load_image_file(caminho_imagem)
    embeddings = face_recognition.face_encodings(imagem)
    vetor = embeddings[0]

    return np.array(vetor)

# Código para selecionar diretórios e separar conjuntos de treino e validação

In [19]:
import os
import random
from sklearn.model_selection import train_test_split

def separa_treino_validacao_lfw(base_dir, proporcao_treino=0.8):
    caminhos_treino = []
    caminhos_validacao = []

    for nome_pessoa in os.listdir(base_dir):
        caminho_pessoa = os.path.join(base_dir, nome_pessoa)

        if not os.path.isdir(caminho_pessoa):
            continue

        imagens = [os.path.join(caminho_pessoa, img) for img in os.listdir(caminho_pessoa)
                   if img.lower().endswith(('.jpg', '.png', '.jpeg'))]

        total_imagens = len(imagens)

        if total_imagens not in [8, 10, 12]:
            continue

        imagens.sort()  # garantir consistência
        random.seed(42)  # garantir que o shuffle será o mesmo em todas as execuções do código
        random.shuffle(imagens)

        treino, validacao = train_test_split(imagens, train_size=proporcao_treino, shuffle=False)

        caminhos_treino.extend(treino)
        caminhos_validacao.extend(validacao)

    return caminhos_treino, caminhos_validacao


# Código para inserir imagem numa coleção 

In [ ]:
def inserir_imagem(nome_colecao, caminho_imagem, vetor_caracteristicas):
    id_ponto = str(uuid.uuid4())    # ID único 

    nome_pessoa = caminho_imagem.split("\\")[-1].split(".")[0]

    # Cria estrutura de ponto para inserir na coleção
    ponto = PointStruct(
            id=id_ponto,
            vector=evtor_caracteristicas.tolist(),
            payload={"nome": nome_pessoa, "arquivo": caminho_imagem})
    
    client.upsert(collection_name=nome_colecao, points=ponto)

# Cria coleções

In [ ]:
def popular_colecoes():
    imagens_treino, imagens_validacao = separa_treino_validacao_lfw("lfw_funneled")

    for imagem in imagens_treino:
        caracteristicas_lbp = extrai_caracteristicas_lbp(imagem)
        caracteristicas_landmarks = extrai_caracteristicas_landmarks(imagem)
        caracteristicas_cnn = extrai_caracteristicas_cnn(imagem)

        inserir_imagem("lbp_treinamento", imagem, caracteristicas_lbp)
        inserir_imagem("landmarks_treinamento", imagem, caracteristicas_landmarks)
        inserir_imagem("cnn_treinamento", imagem, caracteristicas_cnn)

    for imagem in imagens_validacao:
        caracteristicas_lbp = extrai_caracteristicas_lbp(imagem)
        caracteristicas_landmarks = extrai_caracteristicas_landmarks(imagem)
        caracteristicas_cnn = extrai_caracteristicas_cnn(imagem)

        inserir_imagem("lbp_validacao", imagem, caracteristicas_lbp)
        inserir_imagem("landmarks_validacao", imagem, caracteristicas_landmarks)
        inserir_imagem("cnn_validacao", imagem, caracteristicas_cnn)

# Main

In [ ]:
client = cria_cliente()

# Cria coleções para cada método de avaliação
cria_colecao(client, "lbp_treinamento", 54)
cria_colecao(client, "lbp_validacao", 54)
cria_colecao(client, "landmarks_treinamento", 68)
cria_colecao(client, "landmarks_validacao")
cria_colecao(client, "cnn_treinamento", 128)
cria_colecao(client, "cnn_validacao", 128)